In [9]:
import heapq
import matplotlib
from matplotlib import patches
from matplotlib.patches import Rectangle
from matplotlib import animation
from matplotlib.animation import FuncAnimation
import tkinter as tk
from tkinter import ttk
from matplotlib.widgets import Button
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import random
import numpy as np
import itertools
import numpy as np
import gym
from gym import spaces

current_time = 0

class Gridworld:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.walls = []
        self.start = (0, 0)
        self.goal = (width - 1, height - 1)
        self.setup_central_crossroad_and_parking_area()

    def in_bounds(self, id):
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height

    def passable(self, id):
        return id not in self.walls

    def neighbors(self, id):
        (x, y) = id
        results = [(x + 1, y), (x, y - 1), (x - 1, y), (x, y + 1)]
        results = filter(self.in_bounds, results)
        results = filter(self.passable, results)
        return list(results)

    def setup_central_crossroad(self):
        crossroad_size = 4
        mid_x = self.width // 2
        mid_y = self.height // 2
        crossroad_start_x = mid_x - crossroad_size // 2
        crossroad_end_x = crossroad_start_x + crossroad_size
        crossroad_start_y = mid_y - crossroad_size // 2
        crossroad_end_y = crossroad_start_y + crossroad_size
        self.walls = [pos for pos in self.walls if not (crossroad_start_x <= pos[0] < crossroad_end_x and crossroad_start_y <= pos[1] < crossroad_end_y)]

    def setup_central_crossroad_and_parking_area(self):
        crossroad_size = 4
        parking_area_size = 5
        mid_x = self.width // 2
        mid_y = self.height // 2
        self.crossroad_start_x = mid_x - crossroad_size // 2
        self.crossroad_end_x = self.crossroad_start_x + crossroad_size
        self.crossroad_start_y = mid_y - crossroad_size // 2
        self.crossroad_end_y = self.crossroad_start_y + crossroad_size
        self.parking_area_start_x = mid_x - parking_area_size // 2
        self.parking_area_end_x = self.parking_area_start_x + parking_area_size
        self.parking_area_start_y = mid_y - parking_area_size // 2
        self.parking_area_end_y = self.parking_area_start_y + parking_area_size

    def is_in_crossroad(self, pos):
        x, y = pos
        return (self.crossroad_start_x <= x < self.crossroad_end_x and
                self.crossroad_start_y <= y < self.crossroad_end_y)

    def is_in_parking_area(self, pos):
        x, y = pos
        return (self.parking_area_start_x <= x < self.parking_area_end_x and
                self.parking_area_start_y <= y < self.parking_area_end_y)

    def has_vehicle_in_crossroad(self, current_vehicle_pos, vehicles):
        for vehicle in vehicles:
            if vehicle.pos != current_vehicle_pos and self.is_in_crossroad(vehicle.pos):
                return True
        return False

def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

class PriorityQueue:
    def __init__(self):
        self.elements = []

    def empty(self):
        return len(self.elements) == 0

    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))

    def get(self):
        return heapq.heappop(self.elements)[1]

def setup_crossroad_obstacles(grid, obstacle_size, road_width):
    grid.walls = []
    mid_x_start = grid.width // 2 - road_width // 2
    mid_x_end = mid_x_start + road_width
    mid_y_start = grid.height // 2 - road_width // 2
    mid_y_end = mid_y_start + road_width
    for x in range(grid.width):
        for y in range(grid.height):
            in_obstacle = (x < obstacle_size and y < obstacle_size) or \
                          (x < obstacle_size and y >= grid.height - obstacle_size) or \
                          (x >= grid.width - obstacle_size and y < obstacle_size) or \
                          (x >= grid.width - obstacle_size and y >= grid.height - obstacle_size)
            in_road = mid_x_start <= x < mid_x_end or mid_y_start <= y < mid_y_end
            if in_obstacle and not in_road:
                grid.walls.append((x, y))

def draw_grid_with_crossroad(grid, ax):
    ax.set_aspect('equal')
    ax.set_xlim(0, grid.width)
    ax.set_ylim(0, grid.height)
    for x, y in grid.walls:
        ax.fill([x, x + 1, x + 1, x], [y, y, y + 1, y + 1], "black")
    for x in range(grid.width + 1):
        ax.plot([x, x], [0, grid.height], color="gray", lw=0.5)
    for y in range(grid.height + 1):
        ax.plot([0, grid.width], [y, y], color="gray", lw=0.5)
    plt.gca().invert_yaxis()

def setup_roadway_entry_exit_points(grid, road_width):
    mid_x, mid_y = grid.width // 2, grid.height // 2
    lane_offset = road_width // 4
    entry_exit_points = {
        'south_to_north': {
            'start': [(mid_x+1, grid.height - 1)],
            'end': [(mid_x+1, 0)]
        },
        'north_to_south': {
            'start': [(mid_x-1, 0)],
            'end': [(mid_x-1, grid.height - 1)]
        },
        'east_to_west': {
            'start': [(grid.width - 1, mid_y-1)],
            'end': [(0, mid_y-1)]
        },
        'west_to_east': {
            'start': [(0, mid_y+1)],
            'end': [(grid.width - 1, mid_y+1)]
        },
    }
    return entry_exit_points

def distance_between(pos1, pos2):
    return ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5



Vehicle vehicle_south_to_north broadcasting message: {'sender': 'vehicle_south_to_north', 'current_position': (11, 19), 'future_positions': [(11, 19), (11, 18), (11, 17), (11, 16), (11, 15), (11, 14)], 'priority': 1716927221.5099542, 'waiting': False}
Vehicle vehicle_south_to_north broadcasting message: {'sender': 'vehicle_south_to_north', 'current_position': (11, 19), 'future_positions': [(11, 19), (11, 18), (11, 17), (11, 16), (11, 15), (11, 14)], 'priority': 1716927221.5099494, 'waiting': False}
Vehicle vehicle_south_to_north broadcasting message: {'sender': 'vehicle_south_to_north', 'current_position': (11, 18), 'future_positions': [(11, 18), (11, 17), (11, 16), (11, 15), (11, 14), (11, 13)], 'priority': 1716927221.7731128, 'waiting': False}
Vehicle vehicle_south_to_north broadcasting message: {'sender': 'vehicle_south_to_north', 'current_position': (11, 18), 'future_positions': [(11, 18), (11, 17), (11, 16), (11, 15), (11, 14), (11, 13)], 'priority': 1716927221.773111, 'waiting': 


KeyboardInterrupt

